In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Input, Dense, Dropout, Conv1D, MaxPool1D, Flatten, BatchNormalization
from tensorflow.keras import backend as be
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD
import absl.logging
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta, Adam, SGD, RMSprop, Adamax, Nadam
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.initializers import glorot_normal
from keras.constraints import maxnorm
from tensorflow.keras import initializers
import tensorflow as tf
from tensorflow.keras import backend as be
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.layers import TimeDistributed, Reshape
from keras.layers.merge import concatenate
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras import backend as be
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.wrappers.scikit_learn import KerasRegressor


In [ ]:
absl.logging.set_verbosity(absl.logging.ERROR)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Demonstration data used in this excercise is already preprocessed and split into
# training, validation, and test sets

# Use wget to download the data stored in csv format.
import itertools

# Define what files to download; download all of the preprocessed data
# Note that the data are already split into Train, Validation, and Test sets.
# The predictor data are denoted with 'X', the target by 'y'
data_download = {}
data_download["window_size"] = [5, 15]
data_download["data_type"] = ["train", "valid", "test"]
data_download["predictor_or_target"] = ["X", "y"]

# Prepare the combinations of the window sizes and the data types
keys, values = zip(*data_download.items())
data_download_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
display('The kind of data to be downloaded:', data_download_combinations)

print("Downloading started...")
for data_download_param in data_download_combinations:
  file_to_download = "https://frankfurt-school-dataset.s3.eu-central-1.amazonaws.com/Sept2021/window_size_{0}_time_encoding_True/{1}_{2}_window_size_{0}_time_encoding_True.csv"\
                     .format(data_download_param["window_size"], data_download_param["predictor_or_target"], data_download_param["data_type"])

  # the actual downloading
  !wget "$file_to_download"

print("Downloading has finished")

In [ ]:
# Function to load the data with time_window
def load_data(num):

    X_train = pd.read_csv("X_train_window_size_{}_time_encoding_True.csv".format(num))
    y_train = pd.read_csv("y_train_window_size_{}_time_encoding_True.csv".format(num))

    X_valid = pd.read_csv("X_valid_window_size_{}_time_encoding_True.csv".format(num))
    y_valid = pd.read_csv("y_valid_window_size_{}_time_encoding_True.csv".format(num))

    return X_train, y_train, X_valid, y_valid

In [ ]:
time_window = 5
X_train, y_train, X_valid, y_valid = load_data(time_window)

# lets have a look at target variable patter across day (assuming weekdays =7)
df_x = X_train
df_x['target'] = y_train['y']
df_x['weekday'] = np.arcsin(df_x.dlvry_weekday_sin)
df_x['temp'] =df_x.weekday *7 /(2*np.pi)

# Force the histogram to bin the days into 7 bins, though this will not give exact return to 7 days,
# or maybe it does. lets see by giving bins = 10

a, b = np.histogram(df_x.temp,bins=10)
a = np.digitize(df_x.temp, bins=b)
df_x['weekday'] = pd.DataFrame(a)

fig, ax = plt.subplots(figsize=(12,10))
sns.boxplot(df_x.weekday, df_x["target"], ax=ax)
plt.title("Values for weekdays")
plt.xlabel("Weekday")
plt.yscale("symlog")
plt.show()


As Seen even though we tried to put the data in 10 bins we are getting 7 because other bins are empty.

In [ ]:

# lets try to visualize the data for hour, assuming the deliveryhour window = (1,24)
# Again as above we can force the hsitorgram to create more than 24 bins and see what we get.

df_x['hr'] = np.arcsin(df_x.dlvry_hour_sin)
df_x['temp'] =df_x.hr *24 /(2*np.pi)

a, b = np.histogram(df_x.temp,bins=30)
a = np.digitize(df_x.temp, bins=b)
df_x['hr'] = pd.DataFrame(a)

import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,10))
sns.boxplot(df_x.hr, df_x["target"], ax=ax)
plt.title("Values for hours")
plt.xlabel("Hours")
plt.yscale("symlog")
plt.show()

Delivery Hour has lesser number of bins than 24, maybe because of the difference between delivery time window and contract trading hours not sharing same time window.

In [ ]:
# Target variable (contract difference) plot.
plot = y_train["y"].plot()
plt.show()

In [ ]:
# Though its evident from graph that the series is stationary. Lets do the test:

from statsmodels.tsa.stattools import adfuller
result = adfuller(y_train["y"].values)

print('p-value: %f' % result[1])

print("Should be under 0.05")

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# lets look at partial auto correlation for the series first for 5 and then 15.

plot_acf(df_x["target"].values, lags=30 , zero=False)  # method = "ldadjusted"
plt.xlabel("Measurement points back in time - relatively")
plt.yscale("symlog")
plt.show()
plot_pacf(df_x["target"].values, lags=30
, zero=False)
plt.xlabel("Measurement points back in time - relatively")
plt.show()
print("---")

In [ ]:
# PACF and ACF for Time window 15
X_train_15, y_train_15,_,_ = load_data(15)
plt.close()
plot_acf(y_train_15["y"].values, lags=30 , zero=False) 
plt.xlabel("Measurement points back in time - relatively")
# plt.yscale("symlog")
plt.show()
plot_pacf(y_train_15["y"].values, lags=30
, zero=False)
plt.xlabel("Measurement points back in time - relatively")
plt.show()
print("---")

This shows the some correlation than compared to time window 5 series.

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf


# Partial correlation of the Candles columns
# Except for TimeFromLastTradeDiff which will obviously have coorelation other variables have 
# correlation in window of <10 hrs but Volumediff has some jumps beyond this window.

columns = ['0', '1', '2', '3', '4', '5']
list_ = ['OpenDiff', 'HighDiff','LowDiff','CloseDiff','VolumeDiff','TimeFromlasttradeDiff']
for col in columns:
    plt.figure()
    plot_pacf(X_train[col].values, lags=72, zero=False)
    plt.title(list_[int(col)])
    
plt.show()

All these graph tell us that some coorelation is there  in initial hours only but the significance is very less. As per the grpah it looks that the target has very less correlation after the initial hours of 15hrs. As the data by nature of rolling window has the target variable of ith row in the input of ith + 1 row, and since we cannot use this info in any manner. However if this was not case we then we can roll the data with window of 15 hrs and pass this to the models (either as it si or by flattening in case of statistical models)

In [ ]:
def display_history(history,name ="None"):   # As we are interested mainy in Loss we shamelessly copy the code from lecture
                                            # notebook and use it by doing slight modification
    import matplotlib.pyplot as plt
    """Summarize history for accuracy and loss.
    """
    fig, axs = plt.subplots(ncols=1,figsize=(10,6),sharey='row')
    title = "Loss_for_{}".format(name)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    fig.savefig(title + '.png')                       # Save the graphs, if not then please comment out this line.
    plt.show()                                          #it will be saved with default name "None", if no anme passed.

In [ ]:
# Using only Mean Squared error and Mean Absolute error as instructed for model evaluation.

def evaluate_model(model,X_valid, y_valid):
    from math import sqrt
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    predictions = model.predict(X_valid)
    mse = mean_squared_error(y_valid, predictions)
    mae = mean_absolute_error(y_valid, predictions)
    return mse, mae

# Classical Linear Regression Models 

At first, we decided to run some classical linear regression models. They are as follows:
LinearRegression, LassoRegression, RidgeRegression, ElasticNet, PassiveAggresiveRegressor, RANSACRegressor, StochasticGD <br>
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model <br>

**Linear Regression**: <br>
LinearRegression fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

**Lasso Regression**: https://www.mygreatlearning.com/blog/understanding-of-lasso-regression/#introduction <br>
We chose Lasso regression as a technique since it involves usage of fewer parameters and thereby can help in dealing with the multicollinearity problem. It uses shrinkage - i.e. data values are shrunk towards a central point as the mean. 

**Ridge Regression**: <br>
https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Ridge_Regression.pdf <br>
We used Ridge Regression as a technique not only to correct for multicollinearity but also to avoid large variances from the true values. This would also ensure the reduction in standard errors. 

**Elastic Net**: <br>
https://en.wikipedia.org/wiki/Elastic_net_regularization <br>
We used elastic as a regularized regression method since it involves a combination of the Lasso and Ridge methods (it linearly combines the L1 and L2 penalties).

**Passive Aggressive Regression**<br>
https://thecleverprogrammer.com/2021/07/04/passive-aggressive-regression-in-machine-learning/<br>
We used Passive Aggressive Regression algorithm since it can work with real time data and continuously learns new data as it comes. This is particulary suitable for our problem and hence generalizable.

**RANSAC Regression** <br>
https://scikit-learn.org/stable/modules/linear_model.html#ransac-regression<br>
RANSAC is a non-deterministic algorithm producing only a reasonable result with a certain probability, which is dependent on the number of iterations. It is typically used for linear and non-linear regression problems and is especially popular in the field of photogrammetric computer vision.

**SGD Regressor**: <br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html?highlight=sgd%20r#sklearn.linear_model.SGDRegressor <br>
We implemented SGD (Stochastic Gradient Descent) since it is well suited for regression problems with large number of training samples and supports different kinds of loss functions and penalties to fit the models.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor


def get_models():
    
    models=dict()
    models['LinearRegression'] = LinearRegression()
    models['LassoRegression'] = Lasso(alpha = 0.15,random_state = 10) #alpha = 0.15 comes from a grid search!
    models['RidgeRegression'] = Ridge(random_state = 10)
    models['ElasticNet'] = ElasticNet(alpha = 1, l1_ratio = 0.14, random_state = 10) #value of alpha and l1_ratio is based on a grid search on this model!
    models['PassiveAggresive_Reg'] = PassiveAggressiveRegressor(max_iter=1000, tol=1e-3,random_state = 10)
    models['RANSACRegressor'] = RANSACRegressor(random_state = 10)
    models['StochasticGD'] = SGDRegressor(loss = 'epsilon_insensitive', max_iter= 1000,
                                          tol=1e-20, shuffle = False, learning_rate = "adaptive",random_state = 10)
    
    return models

In [ ]:
models = get_models()

report = pd.DataFrame(columns = ['Time_window','model','Val_ms_error', 'Val_ma_error'])  
time_window = [5,15]
for i in time_window:
    X_train, y_train, X_valid, y_valid = load_data(i)
    print("This is for time_window{}".format(i))
    
    for name, model in models.items():
        model.fit(X_train.values,y_train.values.reshape(y_train.shape[0],))   
        mse_error, mae_error = evaluate_model(model,X_valid,y_valid)
        report.loc[len(report)] = [i,name,mse_error,mae_error]
        
        print("MSE on valid for {}: {}".format(name,mse_error))
        print("MAE on valid for {}: {}".format(name,mae_error))
    

In [ ]:
# to see the performance of all 
report 

# Conclusion from Regression models analysis:
We can see from the results that in time window 5 and 15 the two regression models **Elastic Net** and **Lasso Regression** are performing the best among the implemented 7 models.

In [ ]:
# following two cells are a grid search for the value which is written in the get_models function.

In [ ]:

# the value in the actual model is taken from this step!

from numpy import arange
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RepeatedKFold

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define model
ratios = arange(0, 1, 0.01)
alphas = [ 0.0, 1.0, 10.0]
model = ElasticNetCV(l1_ratio=ratios, alphas=alphas, cv=cv, n_jobs=-1)
# fit model
model.fit(X_train,y_train)
# summarize chosen configuration
print('alpha: %f' % model.alpha_)
print('l1_ratio_: %f' % model.l1_ratio_)

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define model
model = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
# fit model
model.fit(X_train, y_train)
# summarize chosen configuration
print('alpha: %f' % model.alpha_)

# CNN Model

In [ ]:
#CNN can extract the feature we have some categorical variable for which it can learn some dependencies on its own
#like the Bankholiday, or Weekday or Hourday for which this contract is.Further by using the FC network we can use 
#these features to extrapolate the target variable.
# lets implement the CNN model. This is a basic model. As can be seen from param grid. Some of the paramters 
# had been selected by doing the grid search which is omitted in analysis but the code for samme can be found 
# the  notebook below. As CNN can extract the feature we have some categorical variable for which it can learn
# some dependencies on its own like the Bankholiday, or Weekday or Hourday for which this contract is.
# deualt vlaue of n_features is kept for time_window =5, which can be overwittten by passing feature value 
# for time_window = 15

def CNN_basemodel(n_features=41):

    from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPool1D, Flatten, AveragePooling1D
    from tensorflow.keras.models import Model
    from tensorflow.keras.optimizers import Adadelta, Adam, SGD
    from tensorflow.keras.losses import sparse_categorical_crossentropy
    from tensorflow.keras.regularizers import l1
    from tensorflow.keras.backend import clear_session
    from tensorflow.keras.callbacks import TensorBoard
    from tensorflow.keras.initializers import glorot_normal
    from keras.constraints import maxnorm
    from tensorflow.keras import initializers
    import tensorflow as tf

    x = Input(shape = (1,n_features))    # data is presented as one row with features set.
    conv1 = Conv1D(filters = 64, kernel_size= 2, strides = 1, padding= "same", activation = 'relu')(x)  # Convoluting using 64 filters
    
    # Pooling layer to select the one value from the filter application on each row in a sliding manner, insetad fo keeping whole grid.
    # We can treat this also as another way of dropout whereby we drop soem information and keep most relevant information.
    pool1 = MaxPool1D(pool_size=1, padding = "same")(conv1)            

    conv2 = Conv1D(filters = 32, kernel_size= 2, strides = 1, padding= "same", activation = 'relu')(pool1) # Convoluting using 32 filters
    # As we dont use pooling here, instead we try a regularization by means of dropout.
    # using second layer of convolution is like using the the learned simple representation of data by filters in previous layer to build 
    # more sophisticated representation
    # Ex: Maybe previous filters learn how price difference interacts with categorical variable like day, holiday, weekend (JUST Yes/NO). 
    # But in next layer it could be learning the order of interaction, is it linear or some non-linear adn order.
    # From analogy of image, Conv1 learns line but in conv2 it uses them to learn about shapes like circle, square etc.
    dropout2 = Dropout(0.2)(conv2)
    drop_flat = Flatten()(dropout2)   # flattening the data to be sent to FC Dense network
    dense1 = Dense(256, activation='relu')(drop_flat)   # we use 256 hidden units of the FC.

    # Acitvation linear is used here because we explicitly want the model to predict the the target variable which can ahve range value (-inf,inf)
    # This is a regression problem, therefore any fucntion which has range space of (-inf,inf) can be used in the last Dense layer
    predictions = Dense(1,activation= 'linear')(dense1)   
    model = Model(inputs = x, outputs = predictions)
    print(model.summary())  # to see the model achitecture and params 
    return model

In [ ]:
# Example to see the architecture of Model
CNN_basemodel()

In [ ]:
# Creating a LSTM model. defualt value of n_features is kept for time_window =5, which can be overwittten by passing feature value 
# for time_window = 15
def LSTM_base(n_features = 41):
    import tensorflow as tf
    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense, Dropout, LSTM, Input
    from tensorflow.keras import backend as be
    from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
    from tensorflow.keras.optimizers import Adam, SGD
    import absl.logging
    from tensorflow.keras.models import Model
    
    x = Input(shape=(1,n_features))   
    # As we are feeding one  LSTM into another we pass the return sequence which is the sequence of hidden states for each time step
    #  rather than last hidden state. Just like CNN explanation above we can imagine similar effect happening with stacked LSTM
     # https://machinelearningmastery.com/stacked-long-short-term-memory-networks/
    # # https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/

    lstm1 = LSTM(units = 100,dropout= 0.2, return_sequences= True)(x)
    lstm2 = LSTM(10)(lstm1)

    # from the LSTM network we feed the output to feed Forward netowrk and use 'linear' activation in last prdiction layer as target is between(-inf,inf)
   
    dense1 = Dense(256, activation='relu')(lstm2)
    predictions = Dense(1, activation='linear')(dense1)
    model = Model(inputs = x, outputs = predictions)
    print(model.summary())
    return model

In [ ]:
# Example to see the architecture of Model
LSTM_base(n_features=50)

In [ ]:

# clear the session
be.clear_session()
tf.compat.v1.reset_default_graph()

# creating variable to save the output from run of each model for each time-window
report = pd.DataFrame(columns = ['epochs','batch_size','optimizer','activation',"Model",'Val_ms_error','Val_ma_error','Time_window'])

Time_windows = [5,15]    # Data given has two set of time_window

# for loop to iterate through both timewindow and 
for i in Time_windows:
    
    batch_size = 100   # keeping batch size 100 for to see how model performs
    epoch = 20   # keeeping epoch low to see if any convergence happening or not.
    X_train, y_train, X_valid, y_valid = load_data(i) # load data set by iterating through Time_window
    n_features = X_train.shape[1]   
    X_train = X_train.values.reshape(X_train.shape[0],1,n_features)  # reshaping the data as LSTM expects data in 3D
    X_valid = X_valid.values.reshape(X_valid.shape[0],1,n_features)   

    model = CNN_basemodel(n_features = n_features)  #calling base model 
    model.compile(optimizer='adam', loss='mse')  # using 'Adam' optimzer , as we did gridsearch 
    # ( on reduced data set, taking X_train[0:30000] as train and X_train[30000:36000] as test) in which Adam seemed to 
    # perform better, though marginally so it doesnt matter as of now.
    history = model.fit(X_train,y_train,validation_data = (X_valid, y_valid), 
                        batch_size = batch_size,epochs=epoch, verbose=1,shuffle = False)
    
    display_history(history,"CNN_BASE")   # display train and val loss fucntion across the epoch
    mse_error, mae_error = evaluate_model(model,X_valid,y_valid)

    # appending results of this iteration to report data frame created to keep track of same. Though we are using the last model state 
    # in the report, by looking at graph we can check if we need to save the best model state. Though in this problem we are not savinng best model
    # as there is very marginal improvment which can be seen from log and grarph.

    report.loc[len(report)] = [epoch,batch_size,"Adam","RELU","CNN_Basic",mse_error,mae_error,i]

    # MAE is more robust to data with outliers and MSE penalizes the outliers a lot more due to value being raised to power(2). 
    print("Mean squared error on valid is: {}".format(mse_error))  # getting both MAE and MSE as required by problem statement.
    print("MAE on valid is: {}".format(mae_error))

    # Calling LSTM model.
    model = LSTM_base(n_features = n_features)
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X_train,y_train,validation_data = (X_valid, y_valid), 
                        batch_size = batch_size,epochs=epoch, verbose=1,shuffle = False)
    display_history(history,"LSTM_BASE")
    mse_error, mae_error = evaluate_model(model,X_valid,y_valid)
    report.loc[len(report)] = [epoch,batch_size,"Adam","RELU","LSTM_Basic",mse_error,mae_error,i]
    print("Mean squared error on valid is: {}".format(mse_error))  
    print("MAE on valid is: {}".format(mae_error))
    

In [ ]:
report.to_csv('LSTM_CNN_BASE.csv')  # to save the report

# Ensemble Models

Having tried the CNN and LSTM we see that dataset is so complex and ahs inertia, thereby lets try to implement Ensemble method like Random forest. Therefore we do the grid search from start for Random forest.

In [ ]:
from sklearn.ensemble import RandomForestRegressor  #try to figure out number of estimators required
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
RForregCV = RandomForestRegressor(random_state=10)
param_grid = {'n_estimators': [100, 200, 300,400], # the number of trees
              'max_depth': [2,4,8],
              'max_features': ['auto', 'log2'] #features in model
              }
X_train, y_train, X_valid, y_valid = load_data(5)
y_train = y_train.values.reshape(y_train.shape[0],)

# we have used the Timeseriesplit as data is temporal in nature and in Cross validation time its important
# to tell split data and use CV in a way which keeps thsi temporal nature intact.
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html
my_cv = TimeSeriesSplit(n_splits=2)   

# mean squsred errror is not provded as scoring method instead it has 'neg_mean_squared_error' which is replacment for MSE
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
CV_rfmodel = GridSearchCV(estimator=RForregCV, param_grid=param_grid, cv = my_cv, scoring = 'neg_mean_squared_error', verbose = 3)
CV_rfmodel.fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
RForregCV = RandomForestRegressor(random_state=10)
#since lowest estimators was better than we use lower estimators and max feature combo, also we see the best result for n_estimators = 100
# therefore we increse the n_estimators value to see if there is further scope fo improvement
param_grid = {'n_estimators': [10, 20,40, 100], 
              'max_depth': [2,4,8,16],
              'max_features': ['auto', 'log2']
              }
X_train, y_train, X_valid, y_valid = load_data(5)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_rfmodel = GridSearchCV(estimator=RForregCV, param_grid=param_grid, cv = my_cv, scoring = 'neg_mean_squared_error', verbose = 3)
CV_rfmodel.fit(X_train, y_train)

In [ ]:
print(CV_rfmodel.best_score_, CV_rfmodel.best_params_)  # getting best params
evaluate_model(CV_rfmodel, X_valid, y_valid)

Best Model for window 5 turned out to be max depth 2 max_features = log2 and n_estimators 10, similarly we do the same with time window 15.


In [ ]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
RForregCV = RandomForestRegressor(random_state=10)
param_grid = {'n_estimators': [100, 200, 300,400], #we try same grid search for 15 window
              'max_depth': [2,4,8,16],
              'max_features': ['auto', 'log2']
              }
X_train, y_train, X_valid, y_valid = load_data(15)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_rfmodel = GridSearchCV(estimator=RForregCV, param_grid=param_grid, cv = my_cv, scoring = 'neg_mean_squared_error', verbose = 3)
CV_rfmodel.fit(X_train, y_train)
print(CV_rfmodel.best_score_, CV_rfmodel.best_params_)
evaluate_model(CV_rfmodel, X_valid, y_valid)

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
RForregCV = RandomForestRegressor(random_state=10)

# from previous analysis of window 5 lower values of n estimators perform better so we try those
# max_depth also we searched on lower side to see if we can get better results.

param_grid = {'n_estimators': [2,3,4,5,6,7],    
              'max_depth': [1,2,3,4],        
              'max_features': ['auto', 'log2']
              }
X_train, y_train, X_valid, y_valid = load_data(15)  # time window 15
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_rfmodel = GridSearchCV(estimator=RForregCV, param_grid=param_grid, cv = my_cv, scoring = 'neg_mean_squared_error', verbose = 3)
CV_rfmodel.fit(X_train, y_train)

In [ ]:
print(CV_rfmodel.best_score_, CV_rfmodel.best_params_)
evaluate_model(CV_rfmodel, X_valid, y_valid)
#BEST MSE for Random forest are seen here. best params best params{'max_depth': 1, 'max_features': 'log2', 'n_estimators': 5}

Random forest has produced good results with low amounts value for n_estimators and max_depth and max_features = log2. We go to XGboost to see if we can improve or not.

In [ ]:
#the first grid search for XG boost with varied max depths of [2,5,12], varied n estimators of [10,50,100,150] and learning rates of [0.1, 0.01,0.05]
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
xgbreg = xgb.XGBRegressor()
param_grid = {'max_depth': [2,5,12], 
              'n_estimators': [10, 50, 100, 150],
              'learning_rate': [0.1, 0.01, 0.05]}
X_train, y_train, X_valid, y_valid = load_data(5)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_xgbmodel = GridSearchCV(estimator=xgbreg, param_grid=param_grid, cv = my_cv, verbose = 2) #n_jobs = -1
CV_xgbmodel.fit(X_train, y_train)


In [ ]:
print(CV_xgbmodel.best_score_, CV_xgbmodel.best_params_)

evaluate_model(CV_xgbmodel, X_valid, y_valid)
#best MAE window 5 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 10}

In [ ]:


xgbreg = xgb.XGBRegressor()
param_grid = {'max_depth': [2,5,12], 
              'n_estimators': [10, 50, 100, 150],
              'learning_rate': [0.1, 0.01, 0.05]}
X_train, y_train, X_valid, y_valid = load_data(15)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_xgbmodel = GridSearchCV(estimator=xgbreg, param_grid=param_grid, cv = my_cv, verbose = 2) #n_jobs = -1
CV_xgbmodel.fit(X_train, y_train)
#the first grid search for XG boost window size 15 with varied max depths of [2,5,12], varied n estimators of [10,50,100,150] and learning rates of [0.1, 0.01,0.05]

In [ ]:
print(CV_xgbmodel.best_score_, CV_xgbmodel.best_params_)
evaluate_model(CV_xgbmodel, X_valid, y_valid)
#The results for the values of learning rate, max_depth and n_estiators are same for both the window sizes
#The values are close to the dummy regressor values
#Best MSE window 15 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 10}

In [ ]:


xgbreg = xgb.XGBRegressor()
param_grid = {'max_depth': [2,3,4,5,6], 
              'n_estimators': [10, 20, 30, 40],
              'learning_rate': [0.1, 0.2, 0.3, 0.4]}
X_train, y_train, X_valid, y_valid = load_data(5)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_xgbmodel = GridSearchCV(estimator=xgbreg, param_grid=param_grid, cv = my_cv, verbose = 2) #n_jobs = -1
CV_xgbmodel.fit(X_train, y_train)
#2nd Grid Search with parameters since we found the highest learning rate of 0.1 from previous grid search to work best we 
# tried values of [0.1, 0.2, 0.3, 0.4] and the n_estimators and max_depth were on the lower side so we try low n_estimator values.

In [ ]:
print(CV_xgbmodel.best_score_, CV_xgbmodel.best_params_) #lower estimators and higher learning rates tried still the best model is the same as previous model

evaluate_model(CV_xgbmodel, X_valid, y_valid)
#The results are similar to grid search 1 so not much improvement best params{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 10}

In [ ]:
#same values as above for window 15

xgbreg = xgb.XGBRegressor()
param_grid = {'max_depth': [2,3,4,5,6], 
              'n_estimators': [10, 20, 30, 40],
              'learning_rate': [0.1, 0.2, 0.3, 0.4]}
X_train, y_train, X_valid, y_valid = load_data(15)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_xgbmodel = GridSearchCV(estimator=xgbreg, param_grid=param_grid, cv = my_cv, verbose = 2) #n_jobs = -1
CV_xgbmodel.fit(X_train, y_train)

In [ ]:
print(CV_xgbmodel.best_score_, CV_xgbmodel.best_params_) #same results as the 1st grid search

evaluate_model(CV_xgbmodel, X_valid, y_valid)
#Best MAE window 15 best params{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 10}

In [ ]:
#Changing the time split in Time Series split

xgbreg = xgb.XGBRegressor()
param_grid = {'max_depth': [2,3,4], 
              'n_estimators': [10, 20],
              'learning_rate': [0.1, 0.2, 0.3],
              }
X_train, y_train, X_valid, y_valid = load_data(5)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=3)
CV_xgbmodel = GridSearchCV(estimator=xgbreg, param_grid=param_grid, cv = my_cv, verbose = 2, n_jobs = -1) #n_jobs = -1
CV_xgbmodel.fit(X_train, y_train)

In [ ]:
print(CV_xgbmodel.best_score_, CV_xgbmodel.best_params_) #with a varied time split still the model is performing a bit worse and doesn't have the best outputs even the mean squared error and mean absolute error increases in the model

evaluate_model(CV_xgbmodel, X_valid, y_valid)

#Even after trying multiple combinations with different learning rates [0.01, 0.01, 0,1, 0.2, 0.3, 0,4] and max depth [2, 3, 4, 5, 6, 12] and [10,20,30,40,50,100, 150] in addition to this different time spits were also tried of 2 and 3.
#the best mae for window size 5 is 2.5 and window size 15 is 3.5 hence, there has not been much improvement with an xg boost model till now.

In [ ]:
#Final Grid Search for xgboost with low number of estimators and low max depths on window 5 as low values are performing great
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
xgbreg = xgb.XGBRegressor()
param_grid = {'max_depth': [2,3,4,5,6], 
              'n_estimators': [2,4,5,6,7,8,9],
              'learning_rate': [0.1, 0.2, 0.3, 0.4]}
X_train, y_train, X_valid, y_valid = load_data(5)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_xgbmodel = GridSearchCV(estimator=xgbreg, param_grid=param_grid, cv = my_cv, verbose = 2) #n_jobs = -1
CV_xgbmodel.fit(X_train, y_train)
print(CV_xgbmodel.best_score_, CV_xgbmodel.best_params_) 

evaluate_model(CV_xgbmodel, X_valid, y_valid)
#Best MSE window 5 best params{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 6}

In [ ]:
#Final Grid Search for xgboost with low number of estimators and low max depths on window 15
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
xgbreg = xgb.XGBRegressor()
param_grid = {'max_depth': [2,3,4,5,6], 
              'n_estimators': [2,4,5,6,7,8,9],
              'learning_rate': [0.1, 0.2, 0.3, 0.4]}
X_train, y_train, X_valid, y_valid = load_data(15)
y_train = y_train.values.reshape(y_train.shape[0],)
my_cv = TimeSeriesSplit(n_splits=2)
CV_xgbmodel = GridSearchCV(estimator=xgbreg, param_grid=param_grid, cv = my_cv, verbose = 2) #n_jobs = -1
CV_xgbmodel.fit(X_train, y_train)
print(CV_xgbmodel.best_score_, CV_xgbmodel.best_params_) 

evaluate_model(CV_xgbmodel, X_valid, y_valid)
#There's a certain trend in XG boost the best performing models have low number of max depth and low number of n_estimators
#and time split is also at its minimum

# LSTM and CNN Models New Method

In [ ]:
# As seen from the above analysis there is no convergence happening, we try to use the limited feature set to see 
# if we can atleast see some convergence 
# behaviour happening. We get columns with [open,high,low,close,vol,timefromlastrade].difference candles i.e
#  columns named [0,1....23] in time_window 5 and [0,1....83] in other dataset by ignoring first 17 columns.
def data_split(X_train, X_valid, time_window):
    X_train = X_train.iloc[:,17:]
    X_valid = X_valid.iloc[:,17:]

    # for time_window we have (time_window -1) candles. We rehape the data to where the candles are stacked on each other
    # [O C H L V DLT ]  - time t
    # [O C H L V DLT ]   - time t+1
    # thereby this becomes a temporal sequence with time steps = Time_window -1, with 6 features
    
    X_train = X_train.values.reshape(X_train.shape[0],time_window-1,6)  
    X_valid = X_valid.values.reshape(X_valid.shape[0],time_window-1,6)
    return X_train, X_valid

In [71]:
# LSTM model with temporal sequence 
# We feed each feature to one LSTM thereby parallezing this and then combine back the LSTM of each sequence.More can be  We flatten this output
# and then feed it to Feedforward FC network. We also try to use different activation fucntion for Dense layer to see if it can break the inertia 
# that we see in the data set so far. We have used 'swish' and 'elu'. (though in notebook it is default 'swish')

def LSTM_limited_model(time_window,n_features):
  n_steps = time_window-1  # for each time window we have (time_window -1) candles this will be the time window value for transformed data set
  # to make it more temporal sequential in inutition.

  set_layers = []  # output layers set from each feature time sequence
  inputs = []    # input layer for each sequence
  for i in range(0,n_features):
    
    input = Input(shape = (n_steps,1)) #transforming shape of data to give to LSTM which excpects atleast 3D the third dimension being batch.
    inputs.append(input)
    layers = LSTM(units=10)(input)
    layers = Flatten()(layers)
    set_layers.append(layers)

  merge = concatenate([x for x in set_layers])  # merge the output from each LSTM sequence
  dense = Dense(100, activation='swish')(merge)
  output = Dense(1, activation = 'linear')(dense)
  model = Model(inputs=[x for x in inputs], outputs=output)  # we have six input sequence
  print(model.summary())
  return model


In [72]:
LSTM_limited_model(5,6)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4, 1)]       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 4, 1)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 4, 1)]       0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 4, 1)]       0                                            
____________________________________________________________________________________________

In [73]:
def CNN_limited_model1(time_window, n_features):
  n_steps = time_window-1
  set_layers = []  # output layers set from each feature time sequence
  inputs = []     # input layer for each sequence

  for i in range(0,n_features): 
    input = Input(shape = (n_steps, 1))
    inputs.append(input)

    # just like LSTM we apply convolution filter to these each sequence
    layers = Conv1D(filters=64, kernel_size=2, activation='swish')(input)
    layers = MaxPool1D(pool_size = 2)(layers)
    layers = Flatten()(layers)
    set_layers.append(layers)


  merge = concatenate([x for x in set_layers])   # merge the output from each sequence
  dense = Dense(100, activation='swish')(merge)
  output = Dense(1)(dense)
  model = Model(inputs=[x for x in inputs], outputs=output) 
  print(model.summary())
  return model   

In [74]:
CNN_limited_model1(5,6)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 4, 1)]       0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 4, 1)]       0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 4, 1)]       0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 4, 1)]       0                                            
____________________________________________________________________________________________

In [ ]:
# report = pd.DataFrame(columns = ['epochs','batch_size','optimizer','activation',"Model",'Val_ms_error','Val_ma_error','Time_window'])

Time_windows = [5,15]
for i in Time_windows:
    n_features = 1
    batch_size = 100

    epoch = 20
    #loading data
    X_train, y_train, X_valid, y_valid = load_data(i)
    X_train, X_valid = data_split(X_train,X_valid,i)  # extracting the candle feature set and then roll the data to make each row sequential

    model = CNN_limited_model1(i,X_train.shape[2])
    model.compile(optimizer='adam', loss='mse')

    # We need to give 6 inputs as there are 6 inputs sequnces so we rehape the data wehereby for timewindow 5 
    # we ahve each row transformed as (24 -> (4,6))
    history = model.fit([X_train[:, :, j].reshape(X_train.shape[0], X_train.shape[1], n_features) for j in range(X_train.shape[2])],
                        y_train,
                        validation_data = ([X_valid[:, :, j].reshape(X_valid.shape[0], X_valid.shape[1], n_features) for j in range(X_valid.shape[2])],
                        y_valid), batch_size = batch_size,epochs=epoch, verbose=1,shuffle = False)

    display_history(history,"CNN_Limited_{}".format(i))
    mse_error, mae_error = evaluate_model(model,X_valid,y_valid)
    report.loc[len(report)] = [epoch,batch_size,"Adam","swish","CNN_Limited",mse_error,mae_error,i]

    model = LSTM_limited_model(i,X_train.shape[2])
    model.compile(optimizer='adam', loss='mse')

    history = model.fit([X_train[:, :, j].reshape(X_train.shape[0], X_train.shape[1], n_features) for j in range(X_train.shape[2])],
                    y_train,
                    validation_data = ([X_valid[:, :, j].reshape(X_valid.shape[0], X_valid.shape[1], n_features) for j in range(X_valid.shape[2])],
                                               y_valid), batch_size = batch_size,epochs=epoch, verbose=1,shuffle = False)

    display_history(history,"LSTM_Limited_{}".format(i))
    mse_error, mae_error = evaluate_model(model,X_valid,y_valid)
    report.loc[len(report)] = [epoch,batch_size,"Adam","swish","LSTM_Limited",mse_error,mae_error,i]

So far with these models we are not able to break the inertia of data set to make any kind of convergence, except some sporadic behaviour we see coming from ELU and Swish for CNN model. For further analysis we increased the epoch while apssing the 'swish' to see any convergence.So far we have not seen better convergence so we finally we decided to merge the CNN and LSTM to try something.

In [76]:
time_window = 5
X_train, y_train, X_valid, y_valid = load_data(time_window)
# we separete the time series data from the categorical feature data set, we will feed the categorical to CNN and 
# the temporal to LSTM which takes input in shape of (n_steps = timewindow -1, features =6) [as explained earlier]. 
# However instead of treating the each time series as independent we treat them dependent and feed to one single LSTM layer.
X_train_embed = X_train.iloc[:,:17]
X_train_time  = X_train.iloc[:,17:]
X_valid_embed = X_valid.iloc[:,:17]
X_valid_time =  X_valid.iloc[:,17:]

X_train_embed = X_train_embed.values.reshape(X_train_embed.shape[0],X_train_embed.shape[1],1) # reshape the data for dimensinal reason of tensorflow.
X_valid_embed = X_valid_embed.values.reshape(X_valid_embed.shape[0],X_valid_embed.shape[1],1)

X_train_time = X_train_time.values.reshape(X_train.shape[0],time_window-1,6) # reshape the data for dimensinal reason of tensorflow
X_valid_time = X_valid_time.values.reshape(X_valid.shape[0],time_window-1,6)
n_features = 6

In [ ]:
# creating report frame
report = pd.DataFrame(columns = ['epochs','batch_size','optimizer','activation','StdDeviation','LSTM1', 'LSTM2', 'learning_rate','weight constraint',"Model",'Val_ms_error','Val_ma_error','Time_window'])

# LSTM + CNN Model

As explained above we runn CNN on categorical data which we can terat as embeddings and time series data to LSTM and then merge them back. We run the LSTM on this again. The idea is LSTM will try to predict the next sequence of the temporal series and from CNN we get feature extraction, we again feed to this to LSTM because this CNN can do only feature extraction it cannot remeber the temporal importance between sequence of data and ehcne we use LSTM again and then feed this to Feedforward to predict the target variable.

1. Recurrent Dropout: Recurrent Dropout is a regularization method for recurrent neural networks. Dropout is applied to the updates to LSTM memory cells (or GRU states), i.e. it drops out the input/update gate in LSTM/GRU

Ref: https://paperswithcode.com/method/recurrent-dropout

2. Batch Normalization: Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. This has the effect of stabilizing the learning process and dramatically reducing the number of training epochs required.
Ref: https://machinelearningmastery.com/batch-normalization-for-training-of-deep-neural-networks/ 

Kernel Initializer: It is used to initialize the initial weights as its importan t for symmetry breaking, as data has such a inertia we try to use this break this inertia.
Through trial and error, we found that normal initializer works best for us with a stddev = 0.5.
Normal Initializer: Parameter: Standard deviation (values of 0.1-1)

Warm Restart: 
Why we used it?
Since most of the outputs our neural network models were stuck in val loss, MSE and MAE values we thought of giving it a warm restart so that it could move out of a local minima it might have been stuck inside and find a new local minima for itself.This effect can be understood with temperature parameter. Initially the system is at higher temperature state and hence more smooth the topology of loss fucntion. At start, the model is able to explore the space more as  it is smooth, however when the temprature reduces (learning rate reduces) the model gets stuck in a confined local space of the loss function and starts finding the local minima.By doing warm restart we again make the toplogy smooth and in a way allow model to go to other part of topology and explore the minima in that confined space. Repeatedly doing this activity we are insured that we end-up finding the better local minima.
It is evidently seen that by using the warm restart by keeping other paprmaters fixed the learning can improve significantly. [paper](https://arxiv.org/pdf/1608.03983.pdf)


Stateful: RNNs can be stateful, which means that they can maintain state across batches during training. That is, the hidden state computed for a (ith element of batch of training data will be used as the initial hidden state for the ith data of next batch of training data. This is set to False by defualt as RNN treats each batch more like a sentence and resets it state for next batch. We tried the same by putting the stateful = True, and making batch size = 1. In temporal continous data we need stateful = True or the batch size should be chosen in a mannner where we see the autocorrelation.
https://machinelearningmastery.com/stateful-stateless-lstm-time-series-forecasting-python/

Different activations we used and why we used different activations and their results?
The best activation function we found by trial and error was ReLU. 
Different out of ordinary activation functions tried:
These gave out some interesting results but dont give convergence.

Weight constraint: A weight constraint is an update to the network that checks the size of the weights, and if the size exceeds a predefined limit, the weights are rescaled so that their size is below the limit or between a range.
Ref: https://machinelearningmastery.com/introduction-to-weight-constraints-to-reduce-generalization-error-in-deep-learning/ 


In [ ]:
batch_size = 100  
lstm_cell1 = 100  # LSTM cell size for first 
epoch = 10
p = 5   # weight constraint variable
stddev = 0.5   # stddev variable to be used for symmetryy breaking while kernel initialization usinf RandomNormal
learning_rate = 0.01   # for optmizer as we are implementing the warm restart with cosine decay we can keep inital value high.
filter1 = 16 
lstmcell2 = 100
rec_dropout = 0.5
# def final_model():

# this takes the temporal data
x1 = Input(shape = (time_window-1,n_features) )   
# with the training period the weights shift which could be their true representaiton, hence we use batch normalization to avoid this effect 
x1 = BatchNormalization()(x1)
lstm1 = LSTM(units = lstm_cell1, bias_initializer = initializers.RandomNormal(stddev= stddev))(x1)  
flat1 = Flatten()(lstm1)

# this takes the categorical/ordinal and rest other from candles
x2 = Input(shape = (X_train_embed.shape[1],1))
conv1 = Conv1D(filters = filter1, kernel_size= 3, strides = 1, padding= "same", activation = 'relu', kernel_constraint=maxnorm(p))(x2)
maxpool1 = MaxPool1D(pool_size = 2)(conv1)
flat2 = Flatten()(maxpool1)
merge = concatenate([flat1,flat2]) #concatenating the outputs from LSTM and CNN
merge = Reshape((228,1))(merge)

# next LSTM layer with features extracted by CNN and LSTM predicting the next eleement of temporal sequence data.
lstm2 = LSTM(units = lstmcell2, recurrent_dropout = rec_dropout, kernel_constraint=maxnorm(p), recurrent_constraint= maxnorm(p),
            kernel_initializer = initializers.RandomNormal(stddev= stddev))(merge)

# lstm2 = LSTM(10)(lstm1)
dense1 = Dense(256, activation='relu')(lstm2)
predictions = Dense(1, activation='linear')(dense1)
model = Model(inputs = ([x1,x2]), outputs = predictions)

model.compile()
print(model.summary())

# implementing warm restart using Cosine decay.
# decay steps in first cycle. So now the first warm restart happens at epoch 3 and then it becomes 1.5*3 and so on...
# # Cosinedecay with restart, decay_steps(cycle = n) = t_mul*decay_steps (cycle = n-1)
first_decay_steps = (int(X_train.shape[0]/batch_size))*3 
lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecayRestarts(learning_rate,
                                        first_decay_steps, t_mul=1.5)
cp1 = ModelCheckpoint("CNN_LSTM"+'/', save_best_only=True, save_format="h5")        
model.compile(optimizer=Adam(learning_rate= lr_decayed_fn), loss='mse')
history = model.fit([X_train_time, X_train_embed],y_train,validation_data = ([X_valid_time,X_valid_embed],y_valid), 
                    batch_size = batch_size,epochs=epoch, verbose=1, callbacks=[cp1], shuffle = False)

display_history(history,"LSTM_CNN")

mse_error, mae_error = evaluate_model(model,[X_valid_time,X_valid_embed],y_valid)
report.loc[len(report)] = [epoch,batch_size,"Adam","RELU",stddev,lstm_cell1,lstmcell2,learning_rate,p,"LSTM_CNN", mse_error,mae_error,5]
print("Mean squared error on valid is: {}".format(mse_error))
print("MAE on valid is: {}".format(mae_error))


In [ ]:
be.clear_session()
tf.compat.v1.reset_default_graph()

# Attention:

EXTRA Code block from grid search and other paramter exploration. this might break if tried to run with this code.as might need soem data manipulation.

In [ ]:
def CNN_model(optimizer='adam', pool_type = 'max', activation = "relu", dropout_rate_1 = None, dropout_rate_2 = None, 
                    filter1 = 64, filter2 = 32, stride1 = (1,1), stride2 = (1,1), learning_rate = 0.001, 
                    stddev =1,weight_constraint =1, kernel1 = (5,5), kernel2 = (2,2), neurons = 256):

    from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPool2D, Flatten, AveragePooling2D
    from tensorflow.keras.models import Model
    from tensorflow.keras.optimizers import Adadelta, Adam, SGD
    from tensorflow.keras.losses import sparse_categorical_crossentropy
    from tensorflow.keras.regularizers import l1
    from tensorflow.keras.backend import clear_session
    from tensorflow.keras.callbacks import TensorBoard
    from tensorflow.keras.initializers import glorot_normal
    from keras.constraints import maxnorm
    from tensorflow.keras import initializers
    import tensorflow as tf


    x = Input(shape = (24,41,1))
    conv1 = Conv2D(filters = filter1, kernel_size= kernel1, strides = stride1, padding= "same", activation = 'relu',
                    kernel_constraint=maxnorm(weight_constraint),kernel_initializer=initializers.RandomNormal(stddev= stddev) )(x)

    if pool_type == 'max':
        pool1 = MaxPool2D(pool_size=(4, 4), padding = "same")(conv1)
    if pool_type == 'average':
        pool1 = AveragePooling2D(pool_size=(4, 4), padding = "same")(conv1)
    dropout1 = Dropout(dropout_rate_1)(pool1)

    conv2 = Conv2D(filters = filter2 , kernel_size= kernel2, strides = stride2, padding= "same", activation = 'relu',
                    kernel_constraint=maxnorm(weight_constraint),kernel_initializer=initializers.RandomNormal(stddev= stddev) )(x)

    dropout2 = Dropout(dropout_rate_2)(conv2)
    drop_flat = Flatten()(dropout2)

    dense1 = Dense(neurons, activation='relu')(drop_flat)
    predictions = Dense(1,activation= 'linear')(dense1)
    model = Model(inputs = x, outputs = predictions)
    model.compile(optimizer="Adam", loss= "mse", metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError()])

    # model.summary()
    return model


In [ ]:
###Code Block we used for grid search###
optimizer = ['Adam','Nadam', 'RMSprop', 'Adadelta']
pool_type = ['max']  , ['average']
filter1=[64]      #   [128,32]
filter2=[32]      #   [32,16]
stride1=[(2,2)]  #[(1,1),(2,2),(3,3)],
stride2 =[(2,2)]   #[(1,1),(2,2),(3,3)],
neurons=[128]
learning_rate =[0.1]#,0.001,0.005],  
weight_constraint = [1,3]  # [1,3]
stddev =[0.5] #[0.1,0.5,1], #[32, 256],
dropout_rate_1 = [ 0.20]#,0.30],
dropout_rate_2= [ 0.30]
kernel1 = [(5,5)]
kernel2 = [(2,2)]
activation = ["relu"]
batch_size = [100] # [32 100]
epochs = [50]  # [50, 100]


for i in batch_size:   # ok
    for j in epochs:    #
        for k in optimizer:#
            for l in learning_rate: #
                for m in dropout_rate_1:  #
                    for n in neurons:  #
                        for o in stddev:  #
                            for p in weight_constraint:    #
                              for q in dropout_rate_2:   #
                                for r in pool_type:
                                  for s in stride1:
                                    for t in stride2:
                                      for u in filter1:
                                        for v in filter2:
                                          for w in kernel1:
                                            for z in kernel2:
                                                be.clear_session()

                                                def schedule(epoch, lr = l): #a decaying learning rate with this function
                                                  if epoch < 5:
                                                    return lr
                                                  else:
                                                    return lr * tf.math.exp(-0.2)
                                                lr_scheduler = LearningRateScheduler(schedule)

                                                model = KerasRegressor(build_fn=  lambda: CNN_model(optimizer=k, pool_type = r, activation = "relu", 
                                                          dropout_rate_1 = m, dropout_rate_2 = q, filter1 = u, 
                                                          filter2 = v, stride1 = s, stride2 = t, learning_rate = l, 
                                                          stddev = o,weight_constraint =p, kernel1 = w, kernel2 = z, 
                                                          neurons = n), epochs=j, batch_size=i, verbose=2, shuffle = False)
                                                history = model.fit(X_train, y_train, batch_size = i,epochs=j, callbacks=[lr_scheduler], shuffle = False)
                                                mse_error, mae_error = evaluate_model(model,X_valid_rolled,y_valid_rolled)
                                                report.loc[len(report)] = [j,i,k,r,"RELU",m,q,u,v,s,t,l,o,p,w,z,n,mse_error,mae_error]
                                                print("Mean squared error on valid is: {}".format(mse_error))
                                                print("MAE on valid is: {}".format(mae_error))

                                  # def fit_lstm(train, batch_size, nb_epoch, neurons):


In [ ]:
# LSTM Grid Model

X_train_rolled, y_train_rolled ,segmenter = data(X_train[:30000],y_train[:30000],time_window=time_window,step=1,forecast_distance=1)
X_valid_rolled, y_valid_rolled, segmenter = data(X_train[30000:36000],y_train[30000:36000],step =1, time_window= time_window, 
                                                            forecast_distance=1)

#                 }     
batch_size = [200]  #  
epochs = [5]        #
optimizer = ['Adam']   # Dont change this
learning_rate = [0.01]             # dont change the this
dropout = [0.3]   
recurrent_dropout = [0.3]                         # 
units = [100]                                     # 
stddev = [1]               # dont change this             
weight_constraint = [3]
stateful = [False]          # dont change this

#                         # 1 weight
# report = pd.DataFrame(columns=['Batch','epoch','optimizer','learning_rate','dropout','units',
#                                 'stddev','weight_constraint', 'Val_ms_error', 'Val_ma_error', 'Time_window','recuurent_dropout','Statefullness',])


# i,j,k,l,m,n,"Default",p,mse_error, mae_error,time_window,q,r
from numpy.random import seed
seed(23)
tf.random.set_seed(1)


for i in batch_size:   # ok
    for j in epochs:    #
        for k in optimizer:#
            for l in learning_rate: #
                for m in dropout:  #
                    for n in units:  #
                        for o in stddev:  #
                            for p in weight_constraint:    #
                              for q in recurrent_dropout:   #
                                for r in stateful:  #

                                  be.clear_session()
                                  def schedule(epoch, lr = l): #a decaying learning rate with this function
                                    if epoch < 5:
                                      return lr
                                    else:
                                      return lr * tf.math.exp(-0.2)
                                  lr_scheduler = LearningRateScheduler(schedule)

                                  # def fit_lstm(train, batch_size, nb_epoch, neurons):

                                  #   model = Sequential()
                                  #   model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
                                  #   model.add(Dense(1))
                                  #   model.compile(loss='mean_squared_error', optimizer='adam')
                                  #   for i in range(nb_epoch):
                                  #     model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
                                  #     model.reset_states()
                                  #   return model

                                  model = Sequential()


                                  model.add(LSTM(units = n, dropout = m, recurrent_dropout = q,input_shape = (time_window,column_count),
                                                  kernel_constraint=maxnorm(p), recurrent_constraint= maxnorm(p), stateful= r,  ))       # batch_input_shape = (i,time_window,column_count)))
                                  model.add(Dense(1,'linear'))
                                  # cp1 = ModelCheckpoint('name'+'/', save_best_only=True, save_format="h5")
                                  model.compile(loss='mean_squared_error', optimizer= k )

                                  # history = model.fit(X_train_rolled, y_train_rolled,validation_data=(X_valid_rolled, y_valid_rolled),epochs=epochs, 
                                  #                     batch_size=batch_size, shuffle = False)
                                  history = model.fit(X_train_rolled, y_train_rolled, batch_size = i,epochs=j, callbacks=[lr_scheduler], shuffle = False)

                                  # predict_model = Sequential([Input(batch_input_shape=(1,...),
                                  # <continue specifying exact same model>])

                                  # model.save_weights('lstm_model.h5')

#                                   predict_model = Sequential()


#                                   predict_model.add(LSTM(units = n, dropout = m, recurrent_dropout = q,
#                                                   kernel_constraint=maxnorm(p), recurrent_constraint= maxnorm(p),stateful= r,batch_input_shape = (i,time_window,column_count)))
#                                   predict_model.add(Dense(1,'linear'))
#                                   # cp1 = ModelCheckpoint('name'+'/', save_best_only=True, save_format="h5")
#                                   model.compile(loss='mean_squared_error', optimizer= k )

#                                   predict_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
#                                   # display_history(history,"RMS(0.0005)_batch_30_cellsize(40)")
                                  mse_error, mae_error = evaluate_model(model,X_valid_rolled,y_valid_rolled)
                                  report.loc[len(report)] = [i,j,k,l,m,n,"Default",p,mse_error, mae_error,time_window,q,r]
                                  print("Mean squared error on valid is: {}".format(mse_error))
                                  print("MAE on valid is: {}".format(mae_error))

                                  ###### Dont ofregt to save the results

In [ ]:
report.to_csv("Grid_LSTM.csv")